In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My\ Drive/tobigs/snorrr2

/content/drive/My Drive/tobigs/snorrr2


In [0]:
# !unzip raw_data.zip -d data/
!pip install pydub

In [0]:
ls

data/  new_snorrr_true_false_light.ipynb  raw_data.zip  weight/


## audio.csv 만들기
(파일 겹치는건 빼기)

In [0]:
cd /content/drive/My\ Drive/tobigs/snorrr2/data

/content/drive/My Drive/tobigs/snorrr2/data


In [0]:
ls

 __MACOSX/                     snoring/
'Male speech, man speaking'/  'Traffic noise, roadway noise'/
'Outside, rural or natural'/   Vehicle/


In [0]:
import pandas as pd
import os

label_info = ['Male speech, man speaking','Outside, rural or natural','snoring','Traffic noise, roadway noise','Vehicle']
df = pd.DataFrame({"file_name":[], "labels":[]})
for label in os.listdir():
  if label=='__MACOSX':
      continue
  label_dir = os.path.join(os.getcwd(), label)
  current_label_num = label_info.index(label)+1
  for file in os.listdir(label_dir):
    if file.endswith("mp3"):
      df = df.append({"file_name":file, "labels":int(current_label_num)} , ignore_index=True)

In [0]:
df.labels = df['labels'].astype(int)

In [0]:
df[df.file_name.duplicated()]['file_name']

1684    20628.mp3
1811    18926.mp3
2111    17375.mp3
2140     1522.mp3
2447     5487.mp3
          ...    
5863     4819.mp3
5888    16218.mp3
5907     2933.mp3
5937    17111.mp3
5944     6003.mp3
Name: file_name, Length: 90, dtype: object

In [0]:
df[df['file_name'].str.find('5487.mp3')!=-1]

,file_name,labels
544,u55487.mp3,1
1263,5487.mp3,1
2447,5487.mp3,2


In [0]:
# 중복되는건 삭제하기
df = df[df['file_name'].isin(df[df.file_name.duplicated()]['file_name'])==False]

In [0]:
len(df)

5772

In [0]:
df.to_csv('audio.csv',index=False)

## 학습하기

In [0]:
cd /content/drive/My\ Drive/tobigs/snorrr2

/content/drive/My Drive/tobigs/snorrr2


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.optim import Adam
# import torchaudio
from torch.utils.data import Dataset, DataLoader
import librosa
import pydub 
from fastprogress import master_bar, progress_bar
import numpy as np
import time
from torchvision.models import *
from torchvision.transforms import transforms
import torch.nn as nn
import os
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings(action="ignore")

path1 = "./data/Male speech, man speaking"
path2 = "./data/Outside, rural or natural"
path3 = "./data/snoring"
path4 = "./data/Traffic noise, roadway noise"
path5 = "./data/Vehicle"

df = pd.read_csv("./data/audio.csv")
# df = df[df["labels"]!=2]
# df = df[df["labels"]!=4]
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42) 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
train_df = train_df.reset_index()
test_df = test_df.reset_index()
print(train_df["labels"].value_counts())

3    1492
1    1021
2     734
5     490
4     302
Name: labels, dtype: int64


In [0]:
from pydub import AudioSegment
def freq_augmentation(y, sr):
    switch = np.random.randint(1,3)
    if(switch==1): #고음
        rate = np.random.uniform(2, 5)
    elif(switch==2): #저음
        rate = np.random.uniform(-5, -2)
    y = librosa.effects.pitch_shift(y, sr, n_steps=rate)
    return y

def adding_random_noise(y, noise_rate=0.005):
    rn = np.random.randn(len(y))
    data_rn = y + noise_rate*rn
    return data_rn

def minus_sound(y):
    temp_numpy = (-1)*y
    return temp_numpy

def shifting_sound(data,  roll_rate=0.3):
    # 그냥 [1, 2, 3, 4] 를 [4, 1, 2, 3]으로 만들어주는건데 이게 효과있는지는 잘 모르겠
    data_roll = np.roll(data, int(len(data) * roll_rate))
    return data_roll

def stretch_sound(data, rate=0.7):
    # stretch 해주는거 비율이 뭐가 좋은지 잘모르겟, 0.8이랑, 1.2랑 차이가 안나는거 같음
    stretch_data = librosa.effects.time_stretch(data, rate)
    return stretch_data

def reverse_sound(data):
    temp_array = []
    for i in range(len(data)):
        temp_array.append(data[len(data)-1-i])
    temp_numpy =np.asarray(temp_array)
    return temp_numpy

def adding_white_noise(file_path, sr=22050, noise_quite=25):
    current_dir = '/content/drive/My Drive/tobigs/snorrr2/'
    noise_dir = os.path.join(current_dir, 'noise')
    sound_file = AudioSegment.from_file(file_path)
    # 길이 만큼 random 돌려서 걔 얻기
    file_list = os.listdir(noise_dir)
    noise_file_idx = np.random.randint(0,len(file_list))
    noise_file = AudioSegment.from_file(os.path.join(noise_dir,file_list[noise_file_idx]))
    noise_file = noise_file-noise_quite
    new_sound = sound_file.overlay(noise_file)
    new_sound.export(os.path.join(current_dir, 'temp.mp3'),format='mp3')
    
    wav,sr = librosa.load(os.path.join(current_dir, 'temp.mp3'),sr=sr)
    return wav

In [0]:
def get_mfcc(file_path, isTrain, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
    #train에만 aug
    if(isTrain==True):
      switch = np.random.randint(0,3)
      wav,sr = librosa.load(file_path,sr=sr)

      # if switch==1:
      #   wav = adding_white_noise(file_path)
      # elif switch==2:
      #   # random한 개수만큼 개수만큼  변형하기!
      #   aug_num = np.random.randint(1,4)
      #   aug_idx_list = np.random.randint(6,size=aug_num)
      #   aug_func_list = [shifting_sound, stretch_sound, reverse_sound, partial(freq_augmentation,sr=sr), adding_random_noise, minus_sound]
      #   for aug_idx in aug_idx_list:
      #     wav = aug_func_list[aug_idx](wav)
    else:
      wav,sr = librosa.load(file_path,sr=sr)

    if wav.shape[0]<5*sr:
        wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:5*sr]
    mfcc = librosa.feature.mfcc(wav, sr, n_mfcc=50) 
    return mfcc

In [0]:
from functools import partial
def get_melspectrogram_db(file_path, isTrain, sr=None, n_fft=2048, hop_length=512, n_mels=128, fmin=20, fmax=8300, top_db=80):
    #train에만 aug
    if(isTrain==True):
      switch = np.random.randint(0,3)
      wav,sr = librosa.load(file_path,sr=sr)

      # if switch==1:
      #   wav = adding_white_noise(file_path)
      # elif switch==2:
      #   # random한 개수만큼 개수만큼  변형하기!
      #   aug_num = np.random.randint(1,4)
      #   aug_idx_list = np.random.randint(6,size=aug_num)
      #   aug_func_list = [shifting_sound, stretch_sound, reverse_sound, partial(freq_augmentation,sr=sr), adding_random_noise, minus_sound]
      #   for aug_idx in aug_idx_list:
      #     wav = aug_func_list[aug_idx](wav)
    else:
      wav,sr = librosa.load(file_path,sr=sr)

    if wav.shape[0]<5*sr:
        wav=np.pad(wav,int(np.ceil((5*sr-wav.shape[0])/2)),mode='reflect')
    else:
        wav=wav[:5*sr]
        
    spec=librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft,hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    return spec_db

def spec_to_image(spec, eps=1e-6):
    # 이미지 자체를 정규화 해서 비슷한 밝기의 spectrogram이 나오도록 함.
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    # spec_min, spec_max = spec_norm.min(), spec_norm.max()
    # spec_scaled = (spec_norm - spec_min) / (spec_max - spec_min)
    # spec_scaled = spec_scaled.astype(np.uint8)
    return spec_norm

In [0]:
# Custom data generator
class TrainDatasetBool(Dataset):
    def __init__(self, df, img_dir, isTrain, transforms=None, use_mfcc=False):
        self.df = df
        self.img_dir = img_dir
        self.transforms = transforms
        self.isTrain = isTrain
        self.use_mfcc = use_mfcc

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # print(self.df["file_name"][idx])

        label = self.df["labels"][idx]
        if label == 1:
            audio_path = os.path.join(path1, self.df["file_name"][idx])
        elif label ==2:
            audio_path = os.path.join(path2, self.df["file_name"][idx])
        elif label == 3:
            audio_path = os.path.join(path3, self.df["file_name"][idx])
        elif label == 4:
            audio_path = os.path.join(path4, self.df["file_name"][idx])
        elif label == 5:
            audio_path = os.path.join(path5, self.df["file_name"][idx])

        ### mfcc 쓸지 mel 쓸지
        if use_mfcc==False:
          image = get_melspectrogram_db(audio_path, self.isTrain)
        else:
          image = get_mfcc(audio_path, self.isTrain)

        image = spec_to_image(image)
        x = []
        x.append(image)
        x = np.array(x)
        image = x.astype("float64")

        # print(image.shape, label, image)
        ### RGB로 바꾸기(Mobilenet에서는 RGB!) (1, 224, 431)->(3, 224, 431)
        image = np.repeat(image,3,axis=0)
        # 3, 50, 431 -> (3, 224, 431)
        image = np.pad(image,((0,0),(0,224-image.shape[1]),(0,0)),mode='constant',constant_values= 0)
        # print(image.shape, label, image)
        # image = torch.tensor(image, device=device).float()
        # cuda로 되어있지 않은듯
        image = torch.tensor(image).float()

        if label ==3:
          label = 1
        else:
          label = 0
        label = label
        # cuda로 되어있지 않은듯??? 안하는 방법이 없나??
        # label = torch.tensor(label, device=device).long()
        label = torch.tensor(label).long()
        

        if self.transforms is not None:
            image = self.transforms(image)
        return image, label

In [0]:
# # Custom data generator
# class TrainDataset(Dataset):
#     def __init__(self, df, img_dir, isTrain, transforms=None, use_mfcc=False):
#         self.df = df
#         self.img_dir = img_dir
#         self.transforms = transforms
#         self.isTrain = isTrain
#         self.use_mfcc = use_mfcc

#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         # print(self.df["file_name"][idx])

#         label = self.df["labels"][idx]
#         if label == 1:
#             audio_path = os.path.join(path1, self.df["file_name"][idx])
#         elif label ==2:
#             audio_path = os.path.join(path2, self.df["file_name"][idx])
#         elif label == 3:
#             audio_path = os.path.join(path3, self.df["file_name"][idx])
#         elif label == 4:
#             audio_path = os.path.join(path4, self.df["file_name"][idx])
#         elif label == 5:
#             audio_path = os.path.join(path5, self.df["file_name"][idx])

#         ### mfcc 쓸지 mel 쓸지
#         if use_mfcc==False:
#           image = get_melspectrogram_db(audio_path, self.isTrain)
#         else:
#           image = get_mfcc(audio_path, self.isTrain)

#         image = spec_to_image(image)
#         x = []
#         x.append(image)
#         x = np.array(x)
#         image = x.astype("float64")

#         # print(image.shape, label)
#         image = torch.tensor(image, device=device).float()
#         label = torch.tensor(label, device=device).long()
       
#         if self.transforms is not None:
#             image = self.transforms(image)
#         return image, label

In [0]:
num_classes = 2
lr = 3e-3
eta_min = 1e-5
t_max = 10
num_epochs = 20
use_mfcc = True
# use_mfcc = False

In [0]:
# model 바꿔보기

# model = torch.hub.load('pytorch/vision:v0.5.0', 'mobilenet_v2', pretrained=True)
model = torch.hub.load('pytorch/vision:v0.5.0', 'mobilenet_v2', pretrained=True)

aug_train = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
aug_test = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# model = resnet18(pretrained=True)
# model.fc = nn.Linear(512,num_classes)
# model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
# model = model.to(device)

# 원래라면 데이터를 전처리 하는 패키지
# aug_train = transforms.Compose([])
# aug_test = transforms.Compose([])



dataset_train = TrainDatasetBool(df=train_df,
                            img_dir="./",
                            transforms=aug_train, isTrain=True, use_mfcc=use_mfcc)

dataset_test = TrainDatasetBool(df=test_df,
                            img_dir="./",
                            transforms=aug_test, isTrain=False, use_mfcc=use_mfcc)


# dataset_train = TrainDataset(df=train_df,
#                             img_dir="./",
#                             transforms=aug_train, isTrain=True, use_mfcc=use_mfcc)

# dataset_test = TrainDataset(df=test_df,
#                             img_dir="./",
#                             transforms=aug_test, isTrain=False, use_mfcc=use_mfcc)

train_loader = DataLoader(dataset=dataset_train, batch_size=64, shuffle=False)
#1576/32(batch 수) -> 50개
test_loader = DataLoader(dataset=dataset_test, batch_size=32, shuffle=False)

print(len(train_loader), len(test_loader))

64 55


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.5.0


In [0]:
# for i, (x_batch, y_batch) in enumerate(train_loader):
#   print(x_batch.shape)

In [0]:
!mkdir '/content/drive/My Drive/tobigs/snorrr2/weight'

mkdir: cannot create directory ‘/content/drive/My Drive/tobigs/snorrr2/weight’: File exists


In [0]:
# model = Classifier(num_classes=num_classes).cuda()
# criterion = nn.BCEWithLogitsLoss().cuda()
# criterion = nn.NLLLoss().cuda()
criterion = nn.CrossEntropyLoss().cuda()
# 모델 전체 params 넣음
optimizer = Adam(params=model.parameters(), lr=lr, amsgrad=False)
#  lr을 cosine 함수의 형태처럼 변화시킨다. lr이 커졌다가 작아졌다가 한다. optimizer 가 변수로 들어가 있음
scheduler = CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    avg_loss = 0.
    print("Epoch : {} ".format(epoch))
    print("Train")
    for i, (x_batch, y_batch) in enumerate(train_loader):
        # x_batch = x_batch.to(device)
        # y_batch = y_batch.to(device)

        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        # https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch
        # gradient 초기화 해줘야 함
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # average loss 구하기
        avg_loss += loss.item() / len(train_loader)
        if i==1:
          print(x_batch.shape)
        if i % 1 == 0:
            _, predicted = torch.max(preds.data, 1)
            # print(y_batch.cpu().numpy(), predicted.cpu().numpy())
            print(i)

    torch.save(model.state_dict(), ("/content/drive/My Drive/tobigs/snorrr2/weight/model"+"_"+str(epoch)+".pth"))
    
    print("Test")
    model.eval()
    valid_preds = np.zeros((len(test_loader), num_classes))
    avg_val_loss = 0.

    real = []
    pred = []
    
    for i, (x_batch, y_batch) in enumerate(test_loader):
        if i>5: # 너무 많이 평가하는듯 해서 줄임
          break
        # x_batch = x_batch.to(device)
        # y_batch = y_batch.to(device)

        preds = model(x_batch)
        _, predicted = torch.max(preds.data, 1)
   
        if i % 100 == 0: 
            print(y_batch.cpu().numpy(), predicted.cpu().numpy())

        real = np.append(real, y_batch.cpu().numpy())
        pred = np.append(pred, predicted.cpu().numpy())

    print(classification_report(real, pred))
    scheduler.step()

Epoch : 0 
Train
0
torch.Size([64, 3, 224, 431])
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
Test
[0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
              precision    recall  f1-score   support

         0.0       0.61      1.00      0.76       117
         1.0       1.00      0.01      0.03        75

    accuracy                           0.61       192
   macro avg       0.81      0.51      0.39       192
weighted avg       0.76      0.61      0.47       192

Epoch : 1 
Train
0
torch.Size([64, 3, 224, 431])
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
Test
[0 0 1 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 1] [0 0 1 

In [0]:
torch.-save(model.state_dict(), ( "/content/drive/My Drive/tobigs/snorrr2/weight/model"+"_"+str(epoch)+".pth"))

## evaluate

In [0]:
model.load_state_dict(torch.load("/content/drive/My Drive/tobigs/snorrr2/weight/model_0.pth"))
model.eval()

<All keys matched successfully>

In [0]:
real = []
pred = []
# batch 개수인 32개씩 
#1576/32 -> 50개 len(test_loader)
for i, (x_batch, y_batch) in enumerate(test_loader):
    if i>5: # 너무 많이 평가하는듯 해서 줄임
      break
    # x_batch = x_batch.to(device)
    # y_batch = y_batch.to(device)
    
    preds = model(x_batch)
    _, predicted = torch.max(preds.data, 1)
    print(i)
    if i % 100 == 0:
        print(y_batch.cpu().numpy()[0], predicted.cpu().numpy()[0])

    real = np.append(real, y_batch.cpu().numpy())
    pred = np.append(pred, predicted.cpu().numpy())
    # real.append(y_batch.cpu().numpy()[0])
    # pred.append(predicted.cpu().numpy()[0])
    
# 10개 class라도 5개 밖에 없었으니까 이런식으로 보여준듯!
print(classification_report(real, pred))

0
1 1
1
2
3
4
5
              precision    recall  f1-score   support

         0.0       0.86      0.99      0.92       121
         1.0       0.98      0.72      0.83        71

    accuracy                           0.89       192
   macro avg       0.92      0.86      0.87       192
weighted avg       0.90      0.89      0.89       192



True False로만 테스트

In [0]:
# print(classification_report(real==3, pred==3))

              precision    recall  f1-score   support

       False       0.94      0.93      0.93       709
        True       0.94      0.95      0.95       867

    accuracy                           0.94      1576
   macro avg       0.94      0.94      0.94      1576
weighted avg       0.94      0.94      0.94      1576



#### balanced Test

In [0]:
label_info = test_df.groupby('labels')
balanced_test_df = label_info.apply(lambda x: x.sample(label_info.size().min())).reset_index(drop=True).drop('index',axis=1)

In [0]:
# train test split 때부터 잘했어야 햇는데 그렇지는 못했으니까 임의로 balance 맞추기 -> test_loader 바꾸기?
dataset_test_balanced = TrainDataset(df=balanced_test_df,
                            img_dir="./",
                            transforms=aug_test, isTrain=False, use_mfcc=use_mfcc)

In [0]:
test_loader_balanced = DataLoader(dataset=dataset_test_balanced, batch_size=30, shuffle=False)

In [0]:
real = []
pred = []
# batch 개수인 32개씩 
#1576/32 -> 50개 len(test_loader)
for i, (x_batch, y_batch) in enumerate(test_loader_balanced):
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)

    preds = model(x_batch)
    _, predicted = torch.max(preds.data, 1)
    print(i)
    if i % 100 == 0:
        print(y_batch.cpu().numpy()[0], predicted.cpu().numpy()[0])

    real = np.append(real, y_batch.cpu().numpy())
    pred = np.append(pred, predicted.cpu().numpy())
    # real.append(y_batch.cpu().numpy()[0])
    # pred.append(predicted.cpu().numpy()[0])
    
# 10개 class라도 5개 밖에 없었으니까 이런식으로 보여준듯!
print(classification_report(real, pred))

0
1 3
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
              precision    recall  f1-score   support

         1.0       0.78      0.82      0.80       123
         2.0       0.58      0.18      0.27       123
         3.0       0.74      0.96      0.83       123
         4.0       0.00      0.00      0.00       123
         5.0       0.36      0.85      0.51       123

    accuracy                           0.56       615
   macro avg       0.49      0.56      0.48       615
weighted avg       0.49      0.56      0.48       615

